In [1]:
import numpy as np
import pandas as pd
import itertools

from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint

In [4]:
data = np.array ([49,58,75,110,112,132,151,276,281,362])

In [9]:
m0 = 200
stat, p_v = stats.wilcoxon(data - m0)

In [11]:
round (p_v,4)

0.2845

In [25]:
wood_data1 = np.array ([22,22,15,13,19,19,18,20,21,13,13,15])
wood_data2 = np.array ([17,18,18,15,12,4,14,15,10])

In [20]:
wood_data1 = np.sort (wood_data1)
wood_data2 = np.sort (wood_data2)

In [26]:
stat2,p_v2 = stats.mannwhitneyu(wood_data1, wood_data2)

In [28]:
round(p_v2,4)

0.029

In [30]:
data_space = pd.read_csv ('challenger.txt', sep = '\t')

In [32]:
data_space.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [46]:
inc_true = data_space [data_space['Incident'] == 1]
inc_true = inc_true[['Temperature']]

In [47]:
inc_false = data_space [data_space['Incident'] == 0]
inc_false = inc_false[['Temperature']]

In [48]:
print inc_true.shape
print inc_false.shape

(7, 1)
(16, 1)


In [50]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [51]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [64]:
tr = np.array (inc_true)
tr = tr.reshape(7)
fl = np.array (inc_false)
fl = fl.reshape (16)

In [70]:
np.random.seed(0)
tr_mean = map (np.mean, get_bootstrap_samples(tr, 1000))
fl_mean = map (np.mean, get_bootstrap_samples(fl, 1000))

In [73]:
delta_mean = map(lambda x: x[1] - x[0], zip(tr_mean, fl_mean))
n_g, v_g = stat_intervals(delta_mean, 0.05)
print n_g, v_g
print round (n_g,4)

1.4504017857142826 8.06457589285714
1.4504


In [74]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [75]:
def get_random_combinations(n1, n2, max_combinations):
    index = range(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [76]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [77]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [80]:
np.random.seed(0)
p_v2= permutation_test(tr, fl, max_permutations = 10000)
print round(p_v2,4)

0.0057
